In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from importlib import reload
import seaborn as sns
import os, sys

sys.path.append(os.path.join("../", "utils"))
import load_data

In [2]:
reload(load_data)
uds = load_data.load_uds()
mri = load_data.load_mri()
csf = load_data.load_csf()
uds_dict, mri_dict = load_data.load_feature_map()
print(uds.shape, mri.shape, csf.shape)

(45100, 94) (11273, 172) (3017, 23)


# Data Preprocessing
## Drop missing
- drop features has more than 50% ppl missing
- drop individuals has more than 50% MRI features missing

## Impute Missing


#### Mean and Mode 

In [3]:
## Drop missing
# - drop features has more than 50% ppl missing
# - drop individuals has more than 50% MRI features missing

def drop_missing(df, row_thres=0.5, col_thres=0.5):
    df = df.loc[df.isna().mean(axis=1)<row_thres, df.isna().mean(axis=0)<col_thres]
    return df

uds = drop_missing(uds)
mri = drop_missing(mri, col_thres=0.6)
csf = drop_missing(csf, col_thres=0.8)

In [ ]:
# Impute missing with column mean (numeric) or mode (categorical)
def impute_missing(df):
    for c in df.columns:
        try:
            if c not in ['NACCID', 'datetime', 'NACCUDSD']:
                if df[c].nunique() <= 20:
                    df[c] = df[c].fillna(value = df[c].mode().values[-1])
                else:
                    df[c] = df[c].fillna(value = df[c].mean())
        except:
            print("Cannot convert column ", c)
    return df

uds = impute_missing(uds)
mri = impute_missing(mri)
csf = impute_missing(csf)
uds = uds.dropna(subset=['NACCUDSD'])
print(Counter(uds['NACCUDSD']))

uds.loc[uds['NACCUDSD']==2, 'NACCUDSD']=1
uds = uds.drop('NACCVNUM', axis=1)
mri = mri[mri['NACCVNUM'] == 1].drop('NACCVNUM', axis=1)
print(uds.shape, mri.shape, csf.shape)

In [5]:
uds

,NACCID,NACCADC,NACCAGE,NACCVNUM,EDUC,SEX,NACCAPOE,NACCUDSD,NACCALZP,MEMORY,...,ANIMALS,VEG,TRAILA,TRAILARR,TRAILALI,TRAILB,TRAILBRR,TRAILBLI,BOSTON,datetime
0,NACC020208,186,69,1,16.0,1,NaN,3,1,1.0,...,16.0,11.0,49.0,0.0,24.0,183.0,1.0,24.0,NaN,2020-06-09
1,NACC107305,186,74,1,18.0,2,NaN,1,8,0.0,...,30.0,16.0,30.0,0.0,24.0,48.0,0.0,24.0,NaN,2021-12-01
2,NACC151065,186,86,1,14.0,2,NaN,3,7,0.0,...,10.0,9.0,64.0,0.0,24.0,300.0,2.0,23.0,NaN,2021-12-21
3,NACC187327,186,68,1,14.0,2,NaN,1,8,0.5,...,22.0,11.0,32.0,1.0,24.0,68.0,0.0,24.0,NaN,2021-09-14
4,NACC188799,186,78,1,14.0,2,NaN,3,7,0.5,...,13.0,10.0,36.0,0.0,24.0,11.0,1.0,24.0,NaN,2022-02-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45095,NACC993286,9661,75,1,21.0,1,1.0,2,7,0.0,...,22.0,15.0,27.0,NaN,NaN,37.0,NaN,NaN,27.0,2006-08-04
45096,NACC994463,9661,66,1,16.0,1,2.0,4,1,1.0,...,27.0,17.0,31.0,0.0,24.0,126.0,0.0,24.0,28.0,2012-01-27
45097,NACC995870,9661,53,1,16.0,2,1.0,1,8,0.0,...,17.0,12.0,16.0,0.0,24.0,44.0,0.0,24.0,NaN,2019-11-21
45098,NACC998475,9661,70,1,13.0,2,1.0,3,7,0.0,...,10.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,20.0,2006-10-20


In [6]:
uds.isna().sum()

NACCID          0
NACCADC         0
NACCAGE         0
NACCVNUM        0
EDUC          294
            ...  
TRAILB       7600
TRAILBRR    19636
TRAILBLI    19640
BOSTON      13857
datetime        0
Length: 75, dtype: int64

### Trying using KNN Hot-deck 

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#don't want to do this for ID, ADC, VNUM, UDSD and datetime
uds_numeric = uds.drop(['NACCID', 'datetime', 'NACCUDSD', 'NACCADC'], axis=1)
uds_nueric = uds_numeric._get_numeric_data()
uds_numeric = pd.DataFrame(scaler.fit_transform(uds_numeric), columns = uds_numeric.columns)
uds_numeric

,NACCAGE,NACCVNUM,EDUC,SEX,NACCAPOE,NACCALZP,MEMORY,ORIENT,JUDGMENT,COMMUN,...,DIGIBLEN,ANIMALS,VEG,TRAILA,TRAILARR,TRAILALI,TRAILB,TRAILBRR,TRAILBLI,BOSTON
0,0.560440,0.0,0.533333,0.0,NaN,0.000000,0.333333,0.166667,0.333333,0.166667,...,NaN,0.207792,0.142857,0.326667,0.000,1.0,0.599315,0.025,1.000000,NaN
1,0.615385,0.0,0.600000,1.0,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,0.389610,0.207792,0.200000,0.000,1.0,0.136986,0.000,1.000000,NaN
2,0.747253,0.0,0.466667,1.0,NaN,0.857143,0.000000,0.000000,0.000000,0.000000,...,NaN,0.129870,0.116883,0.426667,0.000,1.0,1.000000,0.050,0.958333,NaN
3,0.549451,0.0,0.466667,1.0,NaN,1.000000,0.166667,0.000000,0.000000,0.000000,...,NaN,0.285714,0.142857,0.213333,0.025,1.0,0.205479,0.000,1.000000,NaN
4,0.659341,0.0,0.466667,1.0,NaN,0.857143,0.166667,0.166667,0.166667,0.000000,...,NaN,0.168831,0.129870,0.240000,0.000,1.0,0.010274,0.025,1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42886,0.626374,0.0,0.700000,0.0,0.0,0.857143,0.000000,0.000000,0.000000,0.000000,...,0.857143,0.285714,0.194805,0.180000,NaN,NaN,0.099315,NaN,NaN,0.900000
42887,0.527473,0.0,0.533333,0.0,0.2,0.000000,0.333333,0.000000,0.333333,0.000000,...,0.571429,0.350649,0.220779,0.206667,0.000,1.0,0.404110,0.000,1.000000,0.933333
42888,0.384615,0.0,0.533333,1.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,0.220779,0.155844,0.106667,0.000,1.0,0.123288,0.000,1.000000,NaN
42889,0.571429,0.0,0.433333,1.0,0.0,0.857143,0.000000,0.000000,0.000000,0.000000,...,0.428571,0.129870,0.090909,NaN,NaN,NaN,NaN,NaN,NaN,0.666667


In [13]:
from sklearn.impute import KNNImputer as knnimp 

In [17]:
imputer = knnimp(n_neighbors = 5) 
uds_knn =  pd.DataFrame(imputer.fit_transform(uds_numeric),columns = uds_numeric.columns)

In [18]:
uds_knn

,NACCAGE,NACCVNUM,EDUC,SEX,NACCAPOE,NACCALZP,MEMORY,ORIENT,JUDGMENT,COMMUN,...,DIGIBLEN,ANIMALS,VEG,TRAILA,TRAILARR,TRAILALI,TRAILB,TRAILBRR,TRAILBLI,BOSTON
0,0.560440,0.0,0.533333,0.0,0.20,0.000000,0.333333,0.166667,0.333333,0.166667,...,0.657143,0.207792,0.142857,0.326667,0.000,1.000,0.599315,0.025,1.000000,0.853333
1,0.615385,0.0,0.600000,1.0,0.08,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.742857,0.389610,0.207792,0.200000,0.000,1.000,0.136986,0.000,1.000000,0.966667
2,0.747253,0.0,0.466667,1.0,0.08,0.857143,0.000000,0.000000,0.000000,0.000000,...,0.628571,0.129870,0.116883,0.426667,0.000,1.000,1.000000,0.050,0.958333,0.766667
3,0.549451,0.0,0.466667,1.0,0.08,1.000000,0.166667,0.000000,0.000000,0.000000,...,0.657143,0.285714,0.142857,0.213333,0.025,1.000,0.205479,0.000,1.000000,0.806667
4,0.659341,0.0,0.466667,1.0,0.12,0.857143,0.166667,0.166667,0.166667,0.000000,...,0.571429,0.168831,0.129870,0.240000,0.000,1.000,0.010274,0.025,1.000000,0.920000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42886,0.626374,0.0,0.700000,0.0,0.00,0.857143,0.000000,0.000000,0.000000,0.000000,...,0.857143,0.285714,0.194805,0.180000,0.000,1.000,0.099315,0.010,1.000000,0.900000
42887,0.527473,0.0,0.533333,0.0,0.20,0.000000,0.333333,0.000000,0.333333,0.000000,...,0.571429,0.350649,0.220779,0.206667,0.000,1.000,0.404110,0.000,1.000000,0.933333
42888,0.384615,0.0,0.533333,1.0,0.00,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.571429,0.220779,0.155844,0.106667,0.000,1.000,0.123288,0.000,1.000000,0.786667
42889,0.571429,0.0,0.433333,1.0,0.00,0.857143,0.000000,0.000000,0.000000,0.000000,...,0.428571,0.129870,0.090909,0.421333,0.075,0.825,0.427397,0.035,1.000000,0.666667


### Trying EM imputation instead of mean

In [ ]:
def impute_em(X, max_iter = 3000, eps = 1e-08):
    '''(np.array, int, number) -> {str: np.array or int}
    
    Precondition: max_iter >= 1 and eps > 0
    
    Return the dictionary with five keys where:
    - Key 'mu' stores the mean estimate of the imputed data.
    - Key 'Sigma' stores the variance estimate of the imputed data.
    - Key 'X_imputed' stores the imputed data that is mutated from X using 
      the EM algorithm.
    - Key 'C' stores the np.array that specifies the original missing entries
      of X.
    - Key 'iteration' stores the number of iteration used to compute
      'X_imputed' based on max_iter and eps specified.
    '''
    
    nr, nc = X.shape
    C = np.isnan(X) == False
    
    # Collect M_i and O_i's
    one_to_nc = np.arange(1, nc + 1, step = 1)
    M = one_to_nc * (C == False) - 1
    O = one_to_nc * C - 1
    
    # Generate Mu_0 and Sigma_0
    Mu = np.nanmean(X, axis = 0)
    observed_rows = np.where(np.isnan(sum(X.T)) == False)[0]
    S = np.cov(X[observed_rows, ].T)
    if np.isnan(S).any():
        S = np.diag(np.nanvar(X, axis = 0))
    
    # Start updating
    Mu_tilde, S_tilde = {}, {}
    X_tilde = X.copy()
    no_conv = True
    iteration = 0
    while no_conv and iteration < max_iter:
        for i in range(nr):
            S_tilde[i] = np.zeros(nc ** 2).reshape(nc, nc)
            if set(O[i, ]) != set(one_to_nc - 1): # missing component exists
                M_i, O_i = M[i, ][M[i, ] != -1], O[i, ][O[i, ] != -1]
                S_MM = S[np.ix_(M_i, M_i)]
                S_MO = S[np.ix_(M_i, O_i)]
                S_OM = S_MO.T
                S_OO = S[np.ix_(O_i, O_i)]
                Mu_tilde[i] = Mu[np.ix_(M_i)] +\
                    S_MO @ np.linalg.inv(S_OO) @\
                    (X_tilde[i, O_i] - Mu[np.ix_(O_i)])
                X_tilde[i, M_i] = Mu_tilde[i]
                S_MM_O = S_MM - S_MO @ np.linalg.inv(S_OO) @ S_OM
                S_tilde[i][np.ix_(M_i, M_i)] = S_MM_O
        Mu_new = np.mean(X_tilde, axis = 0)
        S_new = np.cov(X_tilde.T, bias = 1) +\
            reduce(np.add, S_tilde.values()) / nr
        no_conv =\
            np.linalg.norm(Mu - Mu_new) >= eps or\
            np.linalg.norm(S - S_new, ord = 2) >= eps
        Mu = Mu_new
        S = S_new
        iteration += 1
    
    result = {
        'mu': Mu,
        'Sigma': S,
        'X_imputed': X_tilde,
        'C': C,
        'iteration': iteration
    }
    
    return result

In [ ]:
# from datetime import datetime as dt
from functools import reduce

In [ ]:
#start = dt.now() This was for a time comparison with R but I don't really care to compare. 
result_imputed = impute_em(uds)
#end = dt.now()

#### Trying using KNN  Hot-deck

In [ ]:
type(uds)

In [ ]:
import impyute as impy
from sklearn.impute import KNNImputer as knnimp 

In [ ]:
nan_euclidean_distances(uds.drop(['NACCID', 'datetime', 'NACCUDSD', 'NACCADC'], axis=1), uds.drop(['NACCID', 'datetime', 'NACCUDSD', 'NACCADC'], axis=1))

In [ ]:
imputer = knnimp(n_neighbors = 6) 
uds_knn = imputer.fit_transform(uds.drop(['NACCID', 'datetime', 'NACCUDSD', 'NACCADC'], axis=1))
uds_knn

# PCA feature dimension reduction

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scaler = StandardScaler()
pca = PCA(n_components=5)

def pca_transform(df, var_dict, pca_thres = 0.8):
    to_ret = pd.DataFrame()
    for cat in var_dict['Category'].unique():
        if cat != 'DEMO':
            var_names = var_dict[var_dict['Category'] == cat]['VariableName'].values
            var_names = set(var_names).intersection(set(df.columns))
            if len(var_names) > 5:
                pca_transformed = pca.fit_transform(scaler.fit_transform(df.loc[:,var_names]))
                num_selected_1 = np.sum(pca.explained_variance_ratio_ > pca_thres)
                num_selected_2 = np.argmax(-np.diff(pca.explained_variance_ratio_, n=1) > 0.1) + 1
                num_selected = max(num_selected_1, num_selected_2)
                temp = pd.DataFrame(pca_transformed[:,:num_selected])
                temp.columns = ["{}_{}".format(cat, i+1) for i in range(num_selected)]
                print(cat, pca.explained_variance_ratio_, num_selected)
                to_ret = pd.concat([to_ret, temp], axis=1)
    return to_ret

In [ ]:
# PCA UDS
uds_pca = pca_transform(uds.drop(['NACCID', 'datetime', 'NACCUDSD', 'NACCADC'], axis=1), uds_dict, pca_thres=0.2)
demo_features = set(uds.columns).intersection(set(uds_dict[uds_dict['Category'] == 'DEMO']['VariableName'].values))
uds_pca = pd.concat([uds_pca, uds[demo_features - set(['NACCADC'])].reset_index(drop=True)], axis=1)
uds_pca['NACCID'] = uds.reset_index()['NACCID']
print(uds_pca.shape)

plt.figure(figsize = (12,8))
sns.heatmap(uds_pca.corr(), annot=True, fmt='.2f', cmap='RdYlGn')

In [ ]:
# PCA MRI data
mri_pca = pca_transform(mri.drop(['NACCID', 'MRIMO', 'MRIDY', 'MRIYR'], axis=1), mri_dict, pca_thres=0.2)
mri_pca['NACCID'] = mri.reset_index()['NACCID']
uds_mri_merged = uds_pca.merge(mri_pca, on='NACCID', how='inner')
print(uds_mri_merged.shape)

plt.figure(figsize = (15, 10))
sns.heatmap(uds_mri_merged.corr(), annot=True, fmt='.2f', cmap='RdYlGn')

In [ ]:
csf_var = csf.drop("NACCADC", axis=1).describe().columns
csf_pca = pca.fit_transform(csf[csf_var])
csf_pca = pd.DataFrame(csf_pca[:,:2])
csf_pca.columns = ['CSF_{}'.format(i) for i in range(csf_pca.shape[1])]
csf_pca['NACCID'] = csf.reset_index()['NACCID']
uds_mri_cfs_merged = uds_mri_merged.merge(csf_pca, on='NACCID', how='inner')
print(uds_mri_cfs_merged.shape)

plt.figure(figsize = (15, 10))
sns.heatmap(uds_mri_cfs_merged.corr(), annot=True, fmt='.2f', cmap='RdYlGn')

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

## UDS Only

In [ ]:
clf = LogisticRegression(random_state=48, max_iter=1000, solver='saga', penalty='elasticnet', C=0.4, l1_ratio=0.3)

X_train, X_test, y_train, y_test = train_test_split(uds_pca.drop(['NACCID', 'NACCALZP', 'NACCUDSD'], axis=1), 
                                                     uds_pca['NACCUDSD'], test_size = 0.3, random_state=48)
clf = clf.fit(X_train, y_train)

print("Training Score:  ")
print(metrics.confusion_matrix(y_train, clf.predict(X_train)))
print(metrics.accuracy_score(y_train, clf.predict(X_train)))
print()
print("Testing Score:  ")
print(metrics.confusion_matrix(y_test, clf.predict(X_test)))
print(metrics.accuracy_score(y_test, clf.predict(X_test)))

## UDS + MRI

In [ ]:
clf = LogisticRegression(random_state=48, max_iter=1000, solver='saga', penalty='elasticnet', C=0.4, l1_ratio=0.3)

X_train, X_test, y_train, y_test = train_test_split(uds_mri_merged.drop(['NACCID', 'NACCALZP', 'NACCUDSD'], axis=1), 
                                                    uds_mri_merged['NACCUDSD'], test_size = 0.3, random_state=48)
clf = clf.fit(X_train, y_train)

print("Training Score:  ")
print(metrics.confusion_matrix(y_train, clf.predict(X_train)))
print(metrics.accuracy_score(y_train, clf.predict(X_train)))
print()
print("Testing Score:  ")
print(metrics.confusion_matrix(y_test, clf.predict(X_test)))
print(metrics.accuracy_score(y_test, clf.predict(X_test)))

## UDS + MRI + CSF

In [ ]:
clf = LogisticRegression(random_state=48, max_iter=2000, solver='saga', penalty='elasticnet', C=0.8, l1_ratio=0.3)

X_train, X_test, y_train, y_test = train_test_split(uds_mri_cfs_merged.drop(['NACCID', 'NACCALZP', 'NACCUDSD'], axis=1), 
                                                    uds_mri_cfs_merged['NACCUDSD'], test_size = 0.2, random_state=48)
clf = clf.fit(X_train, y_train)

print("Training Score:  ")
print(metrics.confusion_matrix(y_train, clf.predict(X_train)))
print(metrics.accuracy_score(y_train, clf.predict(X_train)))
print()
print("Testing Score:  ")
print(metrics.confusion_matrix(y_test, clf.predict(X_test)))
print(metrics.accuracy_score(y_test, clf.predict(X_test)))
